In [1]:
def recall(feature_vectors, feature_labels, rank,test_img_list, gallery_vectors=None, gallery_labels=None):
    num_features = len(feature_labels)
    feature_labels = torch.tensor(feature_labels, device=feature_vectors.device)
    gallery_vectors = feature_vectors if gallery_vectors is None else gallery_vectors

    dist_matrix = torch.cdist(feature_vectors.unsqueeze(0), gallery_vectors.unsqueeze(0)).squeeze(0)
    
    if gallery_labels is None:
        dist_matrix.fill_diagonal_(float('inf'))
        gallery_labels = feature_labels
    else:
        gallery_labels = torch.tensor(gallery_labels, device=feature_vectors.device)

    idx = dist_matrix.topk(k=rank[-1], dim=-1, largest=False)[1]
    acc_list = []
    for r in rank:
        correct = (gallery_labels[idx[:, 0:r]] == feature_labels.unsqueeze(dim=-1)).any(dim=-1).float()
        acc_list.append((torch.sum(correct) / num_features).item())
    return acc_list


In [2]:
def recallWithTask(feature_vectors, feature_labels, rank,test_img_list, gallery_vectors=None, gallery_labels=None):
    num_features = len(feature_labels)
    feature_labels = torch.tensor(feature_labels, device=feature_vectors.device)
    gallery_vectors = feature_vectors if gallery_vectors is None else gallery_vectors

    dist_matrix = torch.cdist(feature_vectors.unsqueeze(0), gallery_vectors.unsqueeze(0)).squeeze(0)
    
    for i in range(len(test_img_list)):
        for j in range(len(test_img_list)):
            index1=test_img_list[i].rfind('\\')
            index2=test_img_list[j].rfind('\\')
            if test_img_list[i][index1+4:index1+7] == test_img_list[j][index2+4:index2+7] and i!=j :
                dist_matrix[i][j]=float('inf')
    
    
    if gallery_labels is None:
        dist_matrix.fill_diagonal_(float('inf'))
        gallery_labels = feature_labels
    else:
        gallery_labels = torch.tensor(gallery_labels, device=feature_vectors.device)

    idx = dist_matrix.topk(k=rank[-1], dim=-1, largest=False)[1]
    acc_list = []
    for r in rank:
        correct = (gallery_labels[idx[:, 0:r]] == feature_labels.unsqueeze(dim=-1)).any(dim=-1).float()
        acc_list.append((torch.sum(correct) / num_features).item())
    return acc_list


In [3]:
def test(net, recall_ids):
    net.eval()
    with torch.no_grad():
        # obtain feature vectors for all data
        for key in eval_dict.keys():
            eval_dict[key]['features'] = []
            imgList=[]
            # for inputs, labels in tqdm(eval_dict[key]['data_loader'], desc='processing {} data'.format(key)):
            print('processing {} data'.format(key))
            for batch_idx, (inputs, labels, imgs) in enumerate(eval_dict[key]['data_loader']):
                inputs, labels = inputs.cuda(), labels.cuda()
                features, classes = net(inputs)
                eval_dict[key]['features'].append(features)
                imgList+=imgs
            eval_dict[key]['features'] = torch.cat(eval_dict[key]['features'], dim=0)

        # compute recall metric
        if data_name == 'isc':
            acc_list = recall(eval_dict['test']['features'], test_data_set.label_list, recall_ids,imgList,
                              eval_dict['gallery']['features'], gallery_data_set.label_list)
        else:
            acc_list = recall(eval_dict['test']['features'], test_data_set.label_list, recall_ids,imgList)
    
    desc = 'Test Epoch {}/{} '.format(epoch, num_epochs)
    for index, rank_id in enumerate(recall_ids):
        desc += 'R@{}:{:.2f}% '.format(rank_id, acc_list[index] * 100)
        results['test_recall@{}'.format(rank_id)].append(acc_list[index] * 100)
    print(desc)
    return acc_list[0]


In [4]:
def testWithTask(net, recall_ids):
    net.eval()
    with torch.no_grad():
        # obtain feature vectors for all data
        for key in eval_dict.keys():
            eval_dict[key]['features'] = []
            imgList=[]
            # for inputs, labels in tqdm(eval_dict[key]['data_loader'], desc='processing {} data'.format(key)):
            print('processing {} data'.format(key))
            for batch_idx, (inputs, labels, imgs) in enumerate(eval_dict[key]['data_loader']):
                inputs, labels = inputs.cuda(), labels.cuda()
                features, classes = net(inputs)
                eval_dict[key]['features'].append(features)
                imgList+=imgs
            eval_dict[key]['features'] = torch.cat(eval_dict[key]['features'], dim=0)

        # compute recall metric
        if data_name == 'isc':
            acc_list = recallWithTask(eval_dict['test']['features'], test_data_set.label_list, recall_ids,imgList,
                              eval_dict['gallery']['features'], gallery_data_set.label_list)
        else:
            acc_list = recallWithTask(eval_dict['test']['features'], test_data_set.label_list, recall_ids,imgList)
    
    desc = 'Test Epoch {}/{} '.format(epoch, num_epochs)
    for index, rank_id in enumerate(recall_ids):
        desc += 'R@{}:{:.2f}%  with task'.format(rank_id, acc_list[index] * 100)
        resultsWithC['test_recall@{}'.format(rank_id)].append(acc_list[index] * 100)
    print(desc)
    return acc_list[0]


In [5]:
import argparse

import pandas as pd
import torch
from thop import profile, clever_format
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from tqdm import tqdm
from dataset import SingleData
from model import Model, set_bn_eval
from utils import  LabelSmoothingCrossEntropyLoss, BatchHardTripletLoss, ImageReader, MPerClassSampler
import numpy as np
import os
import math 
import torchvision.transforms as transforms

def get_data_list(data_path, ratio=0.001):
    img_list = []
    for root, dirs, files in os.walk(data_path):
        if files == []:
            class_name = dirs
        elif dirs == []:
            for f in files:
                img_path = os.path.join(root, f)
                img_list.append(img_path)

    np.random.seed(1)
    train_img_list = np.random.choice(img_list, size=int(len(img_list)*(1-ratio)), replace=False)
    #print(img_list, train_img_list)
    eval_img_list = list(set(img_list) - set(train_img_list))
    ########add
    half=math.floor(len(eval_img_list)/2)
    print(half)
    eval_=eval_img_list[:half]
    test_=eval_img_list[half:]
    #######
    #return class_name, train_img_list, eval_img_list 
    return class_name, train_img_list


def train(net, optim):
    net.train()
    # fix bn on backbone network
    net.apply(set_bn_eval)
    # total_loss, total_correct, total_num, data_bar = 0, 0, 0, tqdm(train_data_loader)
    print("start")
    total_loss, total_correct, total_num, data_bar = 0, 0, 0, enumerate(train_data_loader)
    print("end")
    # for inputs, labels in data_bar:
    for batch_idx, (inputs, labels, _) in data_bar:
        print(batch_idx)
        inputs, labels = inputs.cuda(), labels.cuda()
        features, classes = net(inputs)
        class_loss = class_criterion(classes, labels)
        feature_loss = feature_criterion(features, labels)
        loss = class_loss + feature_loss
        optim.zero_grad()
        loss.backward()
        optim.step()
        pred = torch.argmax(classes, dim=-1)
        total_loss += loss.item() * inputs.size(0)
        total_correct += torch.sum(pred == labels).item()
        total_num += inputs.size(0)
        print('Train Epoch {}/{} - Loss:{:.4f} - Acc:{:.2f}%'.format(epoch, num_epochs, total_loss / total_num, total_correct / total_num * 100))
        # data_bar.set_description('Train Epoch {}/{} - Loss:{:.4f} - Acc:{:.2f}%'
        #                          .format(epoch, num_epochs, total_loss / total_num, total_correct / total_num * 100))

    return total_loss / total_num, total_correct / total_num * 100




if __name__ == '__main__':
    
    
    # parser = argparse.ArgumentParser(description='Train CGD')
    # parser.add_argument('--data_path', default='/home/data', type=str, help='datasets path')
    # parser.add_argument('--data_name', default='car', type=str, choices=['car', 'cub', 'sop', 'isc'],
    #                     help='dataset name')
    # parser.add_argument('--crop_type', default='uncropped', type=str, choices=['uncropped', 'cropped'],
    #                     help='crop data or not, it only works for car or cub dataset')
    # parser.add_argument('--backbone_type', default='resnet50', type=str, choices=['resnet50', 'resnext50'],
    #                     help='backbone network type')
    # parser.add_argument('--gd_config', default='SG', type=str,
    #                     choices=['S', 'M', 'G', 'SM', 'MS', 'SG', 'GS', 'MG', 'GM', 'SMG', 'MSG', 'GSM'],
    #                     help='global descriptors config')
    # parser.add_argument('--feature_dim', default=1536, type=int, help='feature dim')
    # parser.add_argument('--smoothing', default=0.1, type=float, help='smoothing value for label smoothing')
    # parser.add_argument('--temperature', default=0.5, type=float,
    #                     help='temperature scaling used in softmax cross-entropy loss')
    # parser.add_argument('--margin', default=0.1, type=float, help='margin of m for triplet loss')
    # parser.add_argument('--recalls', default='1,2,4,8', type=str, help='selected recall')
    # parser.add_argument('--batch_size', default=128, type=int, help='train batch size')
    # parser.add_argument('--num_epochs', default=20, type=int, help='train epoch number')

    # opt = parser.parse_args()
    # args parse
    data_path="D:\\original_images_5\\few-person\\for colab\\train"
    # data_path="D:\\models\\CGD-master\\dataset\\img"
    test_path="D:\\original_images_5\\few-person\\for colab\\same test\\final extract"
    data_name="CRC"
    crop_type="uncropped"
    backbone_type="resnet50"
    gd_config="GS"
    feature_dim=1536
    smoothing=0.1
    temperature=0.5
    margin=0.1
    tempRecall='1,2,4,8'
    recalls=[int(k) for k in tempRecall.split(',')]
    batch_size=8
    num_epochs=50
    
    class_name, train_img_list = get_data_list(data_path)
    class_test_name, test_img_list = get_data_list(test_path)
    
    train_transform = transforms.Compose([ 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor()
])
    
    
    # data_path, data_name, crop_type, backbone_type = opt.data_path, opt.data_name, opt.crop_type, opt.backbone_type
    # gd_config, feature_dim, smoothing, temperature = opt.gd_config, opt.feature_dim, opt.smoothing, opt.temperature
    # margin, recalls, batch_size = opt.margin, [int(k) for k in opt.recalls.split(',')], opt.batch_size
    # num_epochs = opt.num_epochs
    save_name_pre = '{}_{}_{}_{}_{}_{}_{}_{}_{}_gs_pretrian'.format(data_name, crop_type, backbone_type, gd_config, feature_dim,
                                                        smoothing, temperature, margin, batch_size)

#     results = {'train_loss': [], 'train_accuracy': []}
    resultsWithC = {'train_loss': [], 'train_accuracy': []}
    
    for recall_id in recalls:
#         results['test_recall@{}'.format(recall_id)] = []
        resultsWithC['test_recall@{}'.format(recall_id)] = []
        

    train_data_set=SingleData(class_name, train_img_list, train_transform)
    # dataset loader
    # train_data_set = ImageReader(data_path, data_name, 'train', crop_type)
    # train_sample = MPerClassSampler(train_data_set.labels, batch_size)
    train_sample = MPerClassSampler(train_data_set.label_list, batch_size)
    train_data_loader = DataLoader(train_data_set, batch_sampler=train_sample, num_workers=8)
    # test_data_set = ImageReader(data_path, data_name, 'query' if data_name == 'isc' else 'test', crop_type)
    test_data_set = SingleData(class_test_name, test_img_list, train_transform)
    test_data_loader = DataLoader(test_data_set, batch_size, shuffle=False, num_workers=8)
    eval_dict = {'test': {'data_loader': test_data_loader}}
    if data_name == 'isc':
        gallery_data_set = ImageReader(data_path, data_name, 'gallery', crop_type)
        gallery_data_loader = DataLoader(gallery_data_set, batch_size, shuffle=False, num_workers=8)
        eval_dict['gallery'] = {'data_loader': gallery_data_loader}

    # model setup, model profile, optimizer config and loss definition
    model = Model(backbone_type, gd_config, feature_dim, num_classes=3).cuda()
    # model.load_state_dict(torch.load('D:\\models\\CGD-master_2\\CGD-master\\isc_uncropped_resnet50_GS_1536_0.1_0.5_0.1_128_model.pth'))

    flops, params = profile(model, inputs=(torch.randn(1, 3, 224, 224).cuda(),))
    flops, params = clever_format([flops, params])
    print('# Model Params: {} FLOPs: {}'.format(params, flops))
    optimizer = Adam(model.parameters(), lr=1e-4)
    lr_scheduler = MultiStepLR(optimizer, milestones=[int(0.6 * num_epochs), int(0.8 * num_epochs)], gamma=0.1)
    class_criterion = LabelSmoothingCrossEntropyLoss(smoothing=smoothing, temperature=temperature)
    feature_criterion = BatchHardTripletLoss(margin=margin)

    best_recall = 0.0
    for epoch in range(1, num_epochs + 1):
        train_loss, train_accuracy = train(model, optimizer)
#         results['train_loss'].append(train_loss)
#         results['train_accuracy'].append(train_accuracy)
        resultsWithC['train_loss'].append(train_loss)
        resultsWithC['train_accuracy'].append(train_accuracy)
#         rank = test(model, recalls)
        rank2 = testWithTask(model, recalls)
        
        lr_scheduler.step()

        # save statistics
#         data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
        data_frame_with_condition = pd.DataFrame(data=resultsWithC, index=range(1, epoch + 1))
        
#         data_frame.to_csv('results/{}_statistics.csv'.format(save_name_pre), index_label='epoch')
        data_frame_with_condition.to_csv('results/{}_statistics_with_condition.csv'.format(save_name_pre), index_label='epoch')
        
        # save database and model
        data_base = {}
        if rank2 > best_recall:
            best_recall = rank2
            data_base['test_images'] = test_data_set.img_list
            data_base['test_labels'] = test_data_set.label_list
            data_base['test_features'] = eval_dict['test']['features']
            if data_name == 'isc':
                data_base['gallery_images'] = gallery_data_set.img_list
                data_base['gallery_labels'] = gallery_data_set.label_list
                data_base['gallery_features'] = eval_dict['gallery']['features']
            torch.save(model.state_dict(), 'results/{}_model_with_con.pth'.format(save_name_pre))
            torch.save(data_base, 'results/{}_data_base_with_con.pth'.format(save_name_pre))


C:\Users\Reza\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Reza\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


5
0
['s1', 's2', 's3']
['s1', 's2', 's3']
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'resnet.Bottleneck'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'model.GlobalDescriptor'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'model.L2Norm'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modul

Train Epoch 1/50 - Loss:1.1752 - Acc:50.34%
149
Train Epoch 1/50 - Loss:1.1731 - Acc:50.42%
150
Train Epoch 1/50 - Loss:1.1748 - Acc:50.25%
151
Train Epoch 1/50 - Loss:1.1716 - Acc:50.49%
152
Train Epoch 1/50 - Loss:1.1764 - Acc:50.33%
153
Train Epoch 1/50 - Loss:1.1758 - Acc:50.41%
154
Train Epoch 1/50 - Loss:1.1746 - Acc:50.48%
155
Train Epoch 1/50 - Loss:1.1720 - Acc:50.64%
156
Train Epoch 1/50 - Loss:1.1722 - Acc:50.56%
157
Train Epoch 1/50 - Loss:1.1739 - Acc:50.55%
158
Train Epoch 1/50 - Loss:1.1723 - Acc:50.63%
159
Train Epoch 1/50 - Loss:1.1767 - Acc:50.31%
160
Train Epoch 1/50 - Loss:1.1756 - Acc:50.39%
161
Train Epoch 1/50 - Loss:1.1748 - Acc:50.46%
162
Train Epoch 1/50 - Loss:1.1764 - Acc:50.46%
163
Train Epoch 1/50 - Loss:1.1766 - Acc:50.46%
164
Train Epoch 1/50 - Loss:1.1759 - Acc:50.45%
165
Train Epoch 1/50 - Loss:1.1796 - Acc:50.38%
166
Train Epoch 1/50 - Loss:1.1805 - Acc:50.45%
167
Train Epoch 1/50 - Loss:1.1800 - Acc:50.45%
168
Train Epoch 1/50 - Loss:1.1804 - Acc:50.

Train Epoch 1/50 - Loss:1.1340 - Acc:53.44%
320
Train Epoch 1/50 - Loss:1.1336 - Acc:53.50%
321
Train Epoch 1/50 - Loss:1.1331 - Acc:53.49%
322
Train Epoch 1/50 - Loss:1.1337 - Acc:53.48%
323
Train Epoch 1/50 - Loss:1.1346 - Acc:53.40%
324
Train Epoch 1/50 - Loss:1.1341 - Acc:53.42%
325
Train Epoch 1/50 - Loss:1.1341 - Acc:53.45%
326
Train Epoch 1/50 - Loss:1.1337 - Acc:53.52%
327
Train Epoch 1/50 - Loss:1.1336 - Acc:53.54%
328
Train Epoch 1/50 - Loss:1.1331 - Acc:53.61%
329
Train Epoch 1/50 - Loss:1.1328 - Acc:53.71%
330
Train Epoch 1/50 - Loss:1.1324 - Acc:53.70%
331
Train Epoch 1/50 - Loss:1.1329 - Acc:53.65%
332
Train Epoch 1/50 - Loss:1.1329 - Acc:53.60%
333
Train Epoch 1/50 - Loss:1.1332 - Acc:53.52%
334
Train Epoch 1/50 - Loss:1.1330 - Acc:53.51%
335
Train Epoch 1/50 - Loss:1.1327 - Acc:53.50%
336
Train Epoch 1/50 - Loss:1.1321 - Acc:53.56%
337
Train Epoch 1/50 - Loss:1.1319 - Acc:53.55%
338
Train Epoch 1/50 - Loss:1.1315 - Acc:53.61%
339
Train Epoch 1/50 - Loss:1.1308 - Acc:53.

Train Epoch 1/50 - Loss:1.1247 - Acc:53.77%
491
Train Epoch 1/50 - Loss:1.1245 - Acc:53.73%
492
Train Epoch 1/50 - Loss:1.1241 - Acc:53.73%
493
Train Epoch 1/50 - Loss:1.1239 - Acc:53.69%
494
Train Epoch 1/50 - Loss:1.1237 - Acc:53.69%
495
Train Epoch 1/50 - Loss:1.1242 - Acc:53.63%
496
Train Epoch 1/50 - Loss:1.1236 - Acc:53.70%
497
Train Epoch 1/50 - Loss:1.1238 - Acc:53.69%
498
Train Epoch 1/50 - Loss:1.1235 - Acc:53.66%
499
Train Epoch 1/50 - Loss:1.1241 - Acc:53.65%
500
Train Epoch 1/50 - Loss:1.1236 - Acc:53.67%
501
Train Epoch 1/50 - Loss:1.1236 - Acc:53.64%
502
Train Epoch 1/50 - Loss:1.1236 - Acc:53.68%
503
Train Epoch 1/50 - Loss:1.1238 - Acc:53.67%
504
Train Epoch 1/50 - Loss:1.1244 - Acc:53.64%
505
Train Epoch 1/50 - Loss:1.1247 - Acc:53.61%
506
Train Epoch 1/50 - Loss:1.1241 - Acc:53.67%
507
Train Epoch 1/50 - Loss:1.1240 - Acc:53.67%
508
Train Epoch 1/50 - Loss:1.1240 - Acc:53.68%
509
Train Epoch 1/50 - Loss:1.1236 - Acc:53.73%
510
Train Epoch 1/50 - Loss:1.1234 - Acc:53.

...

Train Epoch 3/50 - Loss:1.0925 - Acc:52.47%
877
Train Epoch 3/50 - Loss:1.0924 - Acc:52.49%
878
Train Epoch 3/50 - Loss:1.0923 - Acc:52.49%
879
Train Epoch 3/50 - Loss:1.0921 - Acc:52.49%
880
Train Epoch 3/50 - Loss:1.0920 - Acc:52.50%
881
Train Epoch 3/50 - Loss:1.0917 - Acc:52.54%
882
Train Epoch 3/50 - Loss:1.0914 - Acc:52.56%
883
Train Epoch 3/50 - Loss:1.0910 - Acc:52.59%
884
Train Epoch 3/50 - Loss:1.0911 - Acc:52.58%
885
Train Epoch 3/50 - Loss:1.0910 - Acc:52.58%
886
Train Epoch 3/50 - Loss:1.0912 - Acc:52.58%
887
Train Epoch 3/50 - Loss:1.0914 - Acc:52.56%
888
Train Epoch 3/50 - Loss:1.0914 - Acc:52.56%
889
Train Epoch 3/50 - Loss:1.0915 - Acc:52.54%
890
Train Epoch 3/50 - Loss:1.0915 - Acc:52.54%
891
Train Epoch 3/50 - Loss:1.0916 - Acc:52.54%
892
Train Epoch 3/50 - Loss:1.0913 - Acc:52.56%
893
Train Epoch 3/50 - Loss:1.0911 - Acc:52.59%
894
Train Epoch 3/50 - Loss:1.0910 - Acc:52.58%
895
Train Epoch 3/50 - Loss:1.0907 - Acc:52.61%
896
Train Epoch 3/50 - Loss:1.0905 - Acc:52.

Train Epoch 3/50 - Loss:1.0856 - Acc:53.46%
1047
Train Epoch 3/50 - Loss:1.0859 - Acc:53.45%
1048
Train Epoch 3/50 - Loss:1.0857 - Acc:53.47%
1049
Train Epoch 3/50 - Loss:1.0858 - Acc:53.46%
1050
Train Epoch 3/50 - Loss:1.0860 - Acc:53.44%
1051
Train Epoch 3/50 - Loss:1.0863 - Acc:53.43%
1052
Train Epoch 3/50 - Loss:1.0863 - Acc:53.45%
1053
Train Epoch 3/50 - Loss:1.0859 - Acc:53.50%
1054
Train Epoch 3/50 - Loss:1.0861 - Acc:53.47%
1055
Train Epoch 3/50 - Loss:1.0858 - Acc:53.48%
1056
Train Epoch 3/50 - Loss:1.0861 - Acc:53.45%
1057
Train Epoch 3/50 - Loss:1.0858 - Acc:53.50%
1058
Train Epoch 3/50 - Loss:1.0860 - Acc:53.51%
1059
Train Epoch 3/50 - Loss:1.0858 - Acc:53.51%
1060
Train Epoch 3/50 - Loss:1.0857 - Acc:53.52%
1061
Train Epoch 3/50 - Loss:1.0855 - Acc:53.54%
1062
Train Epoch 3/50 - Loss:1.0852 - Acc:53.57%
1063
Train Epoch 3/50 - Loss:1.0852 - Acc:53.56%
1064
Train Epoch 3/50 - Loss:1.0849 - Acc:53.59%
1065
Train Epoch 3/50 - Loss:1.0846 - Acc:53.61%
1066
Train Epoch 3/50 - L

Train Epoch 3/50 - Loss:1.0808 - Acc:53.91%
1215
Train Epoch 3/50 - Loss:1.0811 - Acc:53.90%
1216
Train Epoch 3/50 - Loss:1.0809 - Acc:53.92%
1217
Train Epoch 3/50 - Loss:1.0807 - Acc:53.94%
1218
Train Epoch 3/50 - Loss:1.0807 - Acc:53.96%
1219
Train Epoch 3/50 - Loss:1.0806 - Acc:53.97%
1220
Train Epoch 3/50 - Loss:1.0803 - Acc:53.99%
1221
Train Epoch 3/50 - Loss:1.0805 - Acc:53.99%
1222
Train Epoch 3/50 - Loss:1.0806 - Acc:54.00%
1223
Train Epoch 3/50 - Loss:1.0804 - Acc:54.02%
1224
Train Epoch 3/50 - Loss:1.0805 - Acc:54.01%
1225
Train Epoch 3/50 - Loss:1.0803 - Acc:54.01%
1226
Train Epoch 3/50 - Loss:1.0805 - Acc:54.01%
1227
Train Epoch 3/50 - Loss:1.0805 - Acc:54.01%
1228
Train Epoch 3/50 - Loss:1.0805 - Acc:54.01%
1229
Train Epoch 3/50 - Loss:1.0803 - Acc:54.03%
1230
Train Epoch 3/50 - Loss:1.0802 - Acc:54.04%
1231
Train Epoch 3/50 - Loss:1.0800 - Acc:54.06%
1232
Train Epoch 3/50 - Loss:1.0802 - Acc:54.05%
1233
Train Epoch 3/50 - Loss:1.0803 - Acc:54.03%
1234
Train Epoch 3/50 - L

Train Epoch 4/50 - Loss:1.0788 - Acc:54.76%
42
Train Epoch 4/50 - Loss:1.0767 - Acc:54.94%
43
Train Epoch 4/50 - Loss:1.0751 - Acc:54.83%
44
Train Epoch 4/50 - Loss:1.0716 - Acc:55.00%
45
Train Epoch 4/50 - Loss:1.0675 - Acc:55.43%
46
Train Epoch 4/50 - Loss:1.0677 - Acc:55.32%
47
Train Epoch 4/50 - Loss:1.0706 - Acc:55.47%
48
Train Epoch 4/50 - Loss:1.0750 - Acc:54.85%
49
Train Epoch 4/50 - Loss:1.0796 - Acc:54.25%
50
Train Epoch 4/50 - Loss:1.0738 - Acc:54.66%
51
Train Epoch 4/50 - Loss:1.0672 - Acc:55.53%
52
Train Epoch 4/50 - Loss:1.0701 - Acc:55.66%
53
Train Epoch 4/50 - Loss:1.0648 - Acc:56.02%
54
Train Epoch 4/50 - Loss:1.0678 - Acc:55.91%
55
Train Epoch 4/50 - Loss:1.0641 - Acc:56.03%
56
Train Epoch 4/50 - Loss:1.0589 - Acc:56.58%
57
Train Epoch 4/50 - Loss:1.0587 - Acc:56.68%
58
Train Epoch 4/50 - Loss:1.0602 - Acc:56.57%
59
Train Epoch 4/50 - Loss:1.0639 - Acc:56.25%
60
Train Epoch 4/50 - Loss:1.0701 - Acc:56.35%
61
Train Epoch 4/50 - Loss:1.0660 - Acc:56.45%
62
Train Epoch 4

KeyboardInterrupt: 